# Analyse differentielle de midori

In [1]:
from midori64 import *
import midori64_c
from tqdm.notebook import trange, tqdm
import random
from termcolor import colored
import math
import pickle

## Utilities

Adds dark mode to `tqdm` progress bars

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

Determinism is good

In [3]:
random.seed(0xdead)

Utility for drawing heatmaps

In [4]:
def ascii_heatmap(data: list[list[int]], threshold_1=None, threshold_2=None, threshold_3=None) -> str:
    ret = ''

    for row in data:
        for el in row:
            c = '░'
            if threshold_1 is not None and threshold_1(el):
                c = '▒'
            if threshold_2 is not None and threshold_2(el):
                c = '▓'
            if threshold_3 is not None and threshold_3(el):
                c = '█'
            ret += c + ' '
        ret += '\n'

    return ret

## Question 1

Here is an array containing midori's DDT

In [5]:
DDT = [[0 for _ in range(16)] for _ in range(16)]

for x in range(16):
    for y in range(16):
        DDT[x ^ y][S_BOX[x] ^ S_BOX[y]] += 1

DDT

[[16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 4, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0],
 [0, 4, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 4, 2, 2, 2, 0, 0, 0, 2, 0, 2],
 [0, 2, 4, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0],
 [0, 2, 0, 0, 2, 0, 0, 4, 0, 2, 4, 0, 2, 0, 0, 0],
 [0, 2, 0, 4, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 2],
 [0, 0, 0, 2, 0, 4, 2, 0, 0, 0, 0, 2, 0, 4, 2, 0],
 [0, 2, 0, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0],
 [0, 0, 4, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 4, 0, 4],
 [0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 4, 0, 2, 0, 2],
 [0, 0, 4, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0],
 [0, 0, 0, 2, 0, 0, 2, 4, 0, 0, 4, 2, 0, 0, 2, 0],
 [0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 4, 2],
 [0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 4, 2, 0, 0, 2, 4]]

Here's a more visual representation

In [6]:
print(ascii_heatmap(
    DDT,
    threshold_1=lambda x: x == 2,
    threshold_3=lambda x: x >= 4,
))

print('Legend\n\t- ░ >= 0\n\t- ▒ >= 2\n\t- █ >= 4')

█ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ▒ █ ░ ▒ ▒ ▒ ░ ▒ ░ ░ ░ ░ ░ ▒ ░ 
░ █ ░ ░ █ ░ ░ ░ ░ █ ░ ░ █ ░ ░ ░ 
░ ░ ░ ░ ▒ ░ █ ▒ ▒ ▒ ░ ░ ░ ▒ ░ ▒ 
░ ▒ █ ▒ ▒ ▒ ░ ░ ▒ ░ ░ ▒ ░ ░ ░ ░ 
░ ▒ ░ ░ ▒ ░ ░ █ ░ ▒ █ ░ ▒ ░ ░ ░ 
░ ▒ ░ █ ░ ░ ░ ▒ ▒ ░ ░ ░ ▒ ▒ ░ ▒ 
░ ░ ░ ▒ ░ █ ▒ ░ ░ ░ ░ ▒ ░ █ ▒ ░ 
░ ▒ ░ ▒ ▒ ░ ▒ ░ ░ ▒ ░ ▒ ▒ ░ ▒ ░ 
░ ░ █ ▒ ░ ▒ ░ ░ ▒ ▒ ░ ▒ ▒ ░ ░ ░ 
░ ░ ░ ░ ░ █ ░ ░ ░ ░ █ ░ ░ █ ░ █ 
░ ░ ░ ░ ▒ ░ ░ ▒ ▒ ▒ ░ █ ░ ▒ ░ ▒ 
░ ░ █ ░ ░ ▒ ▒ ░ ▒ ▒ ░ ░ ▒ ░ ▒ ░ 
░ ░ ░ ▒ ░ ░ ▒ █ ░ ░ █ ▒ ░ ░ ▒ ░ 
░ ▒ ░ ░ ░ ░ ░ ▒ ▒ ░ ░ ░ ▒ ▒ █ ▒ 
░ ░ ░ ▒ ░ ░ ▒ ░ ░ ░ █ ▒ ░ ░ ▒ █ 

Legend
	- ░ >= 0
	- ▒ >= 2
	- █ >= 4


## Question 2

Find the best differential for this schema

![](./images/png/midori_square_attack5_dark.drawio.png)

Let's consider what happens in the `mixColumn`

$$

\begin{pmatrix}
0 & 1 & 1 & 1 \\
1 & 0 & 1 & 1 \\
1 & 1 & 0 & 1 \\
1 & 1 & 1 & 0 \\
\end{pmatrix}
\begin{pmatrix}
0 \\
0 \\
\delta_1 \\
\delta_2 \\
\end{pmatrix}
=
\begin{pmatrix}
0 \\
0 \\
\delta_2 \\
\delta_1 \\
\end{pmatrix}
$$

To preserve the pattern, we want $\delta_1 = \delta_2$ at the entry of `mixColumns`.

<!-- Because of the key, we want $SB(x) \oplus k_3 = SB(y) \oplus k_4$ -->

All our states share the same none null values, we note this value $\delta_i$ in state $\Delta_i$. The area of this difference is colored in white in the graph.

For example
$$
\Delta_1 = \delta_1 \times
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
\end{pmatrix}
$$

We need to find a _likely_ path where the differences going into `mixColumns`. By looking into our DDT, we notice that the column 2 has many 4 and many zeros. If we fix $\delta_1$ to 2, using the DDT we can calculate $\delta_2$:
- 1 with probability $\left(\frac{1}{4}\right)^2 = 0.0625$, we then get $1 = \delta_3 = \delta_4 = \delta_5$, we can then arrive back to $\delta_6= 2$ with probability $\left(\frac{1}{4}\right)^2 = 0.0625$
- 4 with probability $0.0625$ we then get $4 = \delta_3 = \delta_4 = \delta_5$, we can then arrive back to $\delta_6 = 2$ with probability $0.0625$
- 9 with probability $0.0625$ we then get $9 = \delta_3 = \delta_4 = \delta_5$, we can then arrive back to $\delta_6 = 2$ with probability $0.0625$
- 12 with probability $0.0625$ we then get $12 = \delta_3 = \delta_4 = \delta_5$, we can then arrive back to $\delta_6 = 2$ with probability $0.0625$

This gives us a final probability of
$$
P(\Delta_6) = 4 \times \left(\frac{1}{4}\right)^4 = 1.6\%
$$

## Question 3


Evaluer experimentalement cette probabilité

Here's our modified round with an extra `subCell` at the end

In [7]:
def modified_round(msg: list[int], key: list[int]) -> list[int]:
    ct = subCell(msg)
    ct = shuffleCell(ct)
    ct = mixColumns(ct)
    ct = addRoundKey(ct, key)
    ct = subCell(ct)
    return ct

The pattern we are interested in is
$$
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\delta_\text{out} & 0 & 0 & 0 \\
\delta_\text{out} & 0 & 0 & 0 \\
\end{pmatrix}
$$

In [8]:
def has_pattern(ct1, ct2):
    ret = True
    for i in range(16):
        match i:
            case 2:
                ret = ret and (ct1[2] ^ ct2[2] == ct1[3] ^ ct2[3])
            case 3:
                pass
            case _:
                ret = ret and ct1[i] == ct2[i]
    return ret

Generates random messages and keys and count how often a $\delta_\text{in}, \, \delta_\text{out}$ pair appears.

In [9]:
ITERATIONS = 10_000

score = [[0 for _ in range(16)] for _ in range(16)]

for delta in trange(1, 16):
    for _ in range(ITERATIONS):
        key = [random.randint(0, 15) for _ in range(16)]
        m1 = [random.randint(0, 15) for _ in range(16)]
        m2 = [x for x in m1]
        m2[5] ^= delta
        m2[15] ^= delta


        ct1 = modified_round(m1, key)
        ct2 = modified_round(m2, key)

        if has_pattern(ct1, ct2):
            score[delta][ct1[2] ^ ct2[2]] += 1

  0%|          | 0/15 [00:00<?, ?it/s]

In [11]:
m = max([score[y][x] for x in range(16) for y in range(16)])
probabilities_string = ''

for y in range(len(score)):
    for x in range(len(score[y])):
        if score[y][x] >= 0.8 * m:
            probabilities_string += f'\n\t- delta_in={y} and delta_out={x} probability={score[y][x] / ITERATIONS * 100:.2f}%'


print(ascii_heatmap(score, threshold_3=lambda x: x >= 0.8 * m))
print(f'Found probabilities:{probabilities_string}')

░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ █ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ █ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 
░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ ░ 

Found probabilities:
	- delta_in=2 and delta_out=2 probability=1.42%
	- delta_in=10 and delta_out=10 probability=1.43%


We find a value close to the expected probability of $1.6\%$

## Question 4

In [12]:
random.seed(0xdeadbeef)

keys = [
    [random.randint(0, 15) for _ in range(16)]
    for _ in range(10)
]

This is the diagram of the cipher we want to attack. Light cells are active

![](./images/png/midori_square_attack6.drawio.png)

We first need to retrieve couples $(P, C)$. We can optimize our code here by storing these couples in a hash table indexed by the inactive nibbles of $C$ and containing as values the active nibbles of $P$

In [13]:
def enc(m, ks):
    ct = [x for x in m]
    ct = addRoundKey(ct, ks[0])

    ct = subCell(ct)
    ct = shuffleCell(ct)
    ct = mixColumns(ct)
    ct = addRoundKey(ct, ks[1])

    ct = subCell(ct)
    ct = shuffleCell(ct)
    ct = mixColumns(ct)
    ct = addRoundKey(ct, ks[2])

    ct = subCell(ct)
    ct = shuffleCell(ct)
    ct = mixColumns(ct)
    ct = addRoundKey(ct, ks[3])
    
    ct = subCell(ct)
    ct = addRoundKey(ct , ks[4])
    return ct

Generates a key for the hash table for a given cipher text. We store messages that share the non active nibbles inside of a hash table

In [23]:
def get_key(ct: list[int]) -> str:
    l = []
    for i in [0, 1, 2, 3, 4, 5, 6, 7, 9, 14]:
        l.append(ct[i])
    return str(l)

We can fix a message and modify only certain active nibbles (see figure).

In [24]:
msg = [random.randint(0, 15) for _ in range(16)]

Computing these message ciphers is expensive, so instead these messages can be cached and retrieved from `./midori_data.pkl`.

In [25]:
with open('./midori_data.pkl', 'rb') as f:
    hash_table = pickle.load(f)

In [ ]:
%%script false

hash_table: dict[str, list[int]] = {}

nibbles = [(n1, n2, n3, n4, n5, n6)
           for n1 in range(16)
           for n2 in range(16)
           for n3 in range(16)
           for n4 in range(16)
           for n5 in range(16)
           for n6 in range(16)
           ]

for (n1, n2, n3, n4, n5, n6) in tqdm(nibbles):
    if random.random() < 0.8:
        continue

    m = [x for x in msg]

    m[1] = n1
    m[2] = n2
    m[7] = n3
    m[11] = n4
    m[13] = n5
    m[14] = n6

    # ct = enc(m, keys)
    ct = midori64_c.encrypt_differential(m, keys)

    ct1 = ct[8]
    ct2 = ct[10]
    ct3 = ct[11]
    ct4 = ct[12]
    ct5 = ct[13]
    ct6 = ct[15]

    # if has_pattern_differential(m):
    # couples.append([n1, n2, n3, n4, n5, n6, ct1, ct2, ct3, ct4, ct5, ct6])
    delta = get_key(ct)
    if delta in hash_table:
        hash_table[delta].append([n1, n2, n3, n4, n5, n6, ct1, ct2, ct3, ct4, ct5, ct6])
    else:
        hash_table[delta] = [[n1, n2, n3, n4, n5, n6, ct1, ct2, ct3, ct4, ct5, ct6]]

with open('midori_data.pkl', 'wb') as f:
    pickle.dump(hash_table, f)

This function lists valid key nibbles for a given message pair

In [26]:
def get_key_guess(n1: list[int], n2: list[int]) -> [str]:

    for i in range(12):
        if n1[i] == n2[i]:
            return []

    ret = []

    for k1 in range(16):
        if S_BOX[n1[0] ^ k1] ^ S_BOX[n2[0] ^ k1] != 2:
            continue

        for k2 in range(16):
            if S_BOX[n1[1] ^ k2] ^ S_BOX[n2[1] ^ k2] != 2:
                continue

            for k3 in range(16):
                if S_BOX[n1[2] ^ k3] ^ S_BOX[n2[2] ^ k3] != 2:
                    continue

                for k4 in range(16):
                    if S_BOX[n1[3] ^ k4] ^ S_BOX[n2[3] ^ k4] != 2:
                        continue

                    for k5 in range(16):
                        if S_BOX[n1[4] ^ k5] ^ S_BOX[n2[4] ^ k5] != 2:
                            continue

                        for k6 in range(16):
                            if S_BOX[n1[5] ^ k6] ^ S_BOX[n2[5] ^ k6] != 2:
                                continue

                            for k7 in range(16):
                                if S_BOX[n1[6] ^ k7] ^ S_BOX[n2[6] ^ k7] != 2:
                                    continue

                                for k8 in range(16):
                                    if S_BOX[n1[7] ^ k8] ^ S_BOX[n2[7] ^ k8] != 2:
                                        continue

                                    for k9 in range(16):
                                        if S_BOX[n1[8] ^ k9] ^ S_BOX[n2[8] ^ k9] != 2:
                                            continue

                                        for k10 in range(16):
                                            if S_BOX[n1[9] ^ k10] ^ S_BOX[n2[9] ^ k10] != 2:
                                                continue

                                            for k11 in range(16):
                                                if S_BOX[n1[10] ^ k11] ^ S_BOX[n2[10] ^ k11] != 2:
                                                    continue

                                                for k12 in range(16):
                                                    if S_BOX[n1[11] ^ k12] ^ S_BOX[n2[11] ^ k12] != 2:
                                                        continue
                                                    ret.append(str([k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, k12]))
                                                    # return ret

    return ret

We now find valid key pairs for each message pair. For performance reasons it might be important to stop the script when the program `"Found a key pair with 2 guesses"`

In [37]:
key_guesses = {}

threshold = 1

hash_table_keys = list(hash_table.keys())

random.shuffle(hash_table_keys)

for nibbles in tqdm(hash_table_keys):
    nibs = hash_table[nibbles]
    for i in range(len(nibs)):
        for j in range(i + 1, len(nibs)):
            n1 = nibs[i]
            n2 = nibs[j]

            key_guess_arr = get_key_guess(n1, n2)

            for key_guess in key_guess_arr:
                if key_guess in key_guesses:
                    if key_guesses[key_guess] >= threshold:
                        print(f'Found a key pair with {threshold + 1} guesses')
                        threshold += 1

                    key_guesses[key_guess] += 1
                else:
                    key_guesses[key_guess] = 1

  0%|          | 0/1018972 [00:00<?, ?it/s]

Found a key pair with 1 guesses
Found a key pair with 2 guesses


KeyboardInterrupt: 

The entry for the correct message

In [38]:
CORRECT_KEY_ENTRY = str([keys[0][1], keys[0][2], keys[0][7], keys[0][11], keys[0][13], keys[0][14], keys[4][8], keys[4][10], keys[4][11], keys[4][12], keys[4][13], keys[4][15]])
CORRECT_KEY_ENTRY

'[13, 15, 12, 8, 8, 8, 4, 15, 3, 13, 13, 14]'

We then find how many times this ke guess appeared

In [39]:
CORRECT_KEY_GUESSES = key_guesses['[13, 15, 12, 8, 8, 8, 4, 15, 3, 13, 13, 14]']
CORRECT_KEY_GUESSES

3

We then look at how many keys we guessed.

In [40]:
m, GUESSED_KEY = 1, ''
count = 0
stat2 = 0

for k in key_guesses:
    count += key_guesses[k]

    if key_guesses[k] == m:
        stat2 += 1

    if key_guesses[k] > m:
        m = key_guesses[k]
        GUESSED_KEY = k
        stat2 = 1


print(f'The maximum amount of times a key was guessed was {m}\nThe correct key was guessed {CORRECT_KEY_GUESSES} times\nOut of the 2^{int(math.log2(count))} key guesses, 2^{int(math.log2(stat2))} reached the maximum value')

The maximum amount of times a key was guessed was 3
 The correct key was guessed 3 times
Out of the 2^26 key guesses, 2^13 reached the maximum value


(3, '[13, 13, 12, 8, 8, 8, 4, 13, 1, 13, 4, 12]', 109063947, 8192)